In [1]:
# !pip install torch_geometric

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import torch
import networkx as nx

In [2]:
train_data = pd.read_csv('train_stock_data.csv')
train_data['Date'] = pd.to_datetime(train_data['Date'], format='%Y-%m-%d')
train_data.sort_values(['Ticker', 'Date'], inplace=True)
train_data

,Date,Ticker,Open,Open.1,Open.2,Open.3,Open.4,Open.5,Open.6,Open.7,...,Volume.10,Volume.11,Volume.12,Volume.13,Volume.14,Volume.15,Volume.16,Volume.17,Volume.18,Volume.19
0,2019-01-02,AAPL,36.944462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-03,AAPL,34.342203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-04,AAPL,34.473390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-07,AAPL,35.468021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-08,AAPL,35.673153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11083,2022-12-23,XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11539400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11084,2022-12-27,XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11962100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11085,2022-12-28,XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10702100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11086,2022-12-29,XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10534000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
validation_data = pd.read_csv('validation_stock_data.csv')
with open('hyperedges.json', 'r') as f:
    hyperedges = json.load(f)

In [4]:
l = []
tickers = train_data['Ticker'].unique()
for ticker in tickers:
    data = train_data[train_data["Ticker"] == ticker]
    valid_cols = []
    for i in data.columns:
        if data[i].isna().sum() == 0:
            valid_cols.append(i)
    a = data[valid_cols].values
    for i in a:
        l.append(i)
df = pd.DataFrame(l, columns=["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"])
df

,Date,Ticker,Open,High,Low,Close,Volume
0,2019-01-02,AAPL,36.944462,37.889005,36.787037,37.667179,148158800.0
1,2019-01-03,AAPL,34.342203,34.757230,33.869933,33.915253,365248800.0
2,2019-01-04,AAPL,34.473390,35.432244,34.299271,35.363071,234428400.0
3,2019-01-07,AAPL,35.468021,35.499030,34.800162,35.284359,219111200.0
4,2019-01-08,AAPL,35.673153,36.212212,35.425093,35.956993,164101200.0
...,...,...,...,...,...,...,...
20155,2022-12-23,XOM,99.121032,100.780001,99.074697,100.724396,11539400.0
20156,2022-12-27,XOM,101.271219,102.383381,100.863433,102.123878,11962100.0
20157,2022-12-28,XOM,101.864365,101.947773,100.001499,100.446358,10702100.0
20158,2022-12-29,XOM,100.084912,101.688276,100.084912,101.206337,10534000.0


In [5]:
from sklearn.preprocessing import RobustScaler 
scaler = RobustScaler()
scaler1 = RobustScaler()
feature_cols = ['Open', 'High', 'Low', 'Close']
df[feature_cols] = scaler.fit_transform(df[feature_cols])
df["Volume"] = scaler1.fit_transform(df[["Volume"]])
df

,Date,Ticker,Open,High,Low,Close,Volume
0,2019-01-02,AAPL,-0.462705,-0.458112,-0.460747,-0.456032,5.101136
1,2019-01-03,AAPL,-0.493476,-0.494767,-0.495698,-0.500455,13.554442
2,2019-01-04,AAPL,-0.491925,-0.486867,-0.490554,-0.483313,8.460403
3,2019-01-07,AAPL,-0.480164,-0.486085,-0.484553,-0.484245,7.863964
4,2019-01-08,AAPL,-0.477738,-0.477738,-0.477065,-0.476281,5.721920
...,...,...,...,...,...,...,...
20155,2022-12-23,XOM,0.272531,0.277970,0.285540,0.290567,-0.218712
20156,2022-12-27,XOM,0.297957,0.296736,0.306972,0.307137,-0.202252
20157,2022-12-28,XOM,0.304971,0.291638,0.296645,0.287275,-0.251315
20158,2022-12-29,XOM,0.283929,0.288601,0.297644,0.296273,-0.257861


In [28]:
val_data = pd.read_csv('validation_stock_data.csv')
val_data['Date'] = pd.to_datetime(val_data['Date'], format='%Y-%m-%d')
val_data.sort_values(['Ticker', 'Date'], inplace=True)
l = []
val_tickers = sorted(val_data['Ticker'].unique())
for ticker in val_tickers:
    data = val_data[val_data["Ticker"] == ticker]
    valid_cols = []
    for i in data.columns:
        if data[i].isna().sum() == 0:
            valid_cols.append(i)
    a = data[valid_cols].values
    for i in a:
        l.append(i)
df1 = pd.DataFrame(l, columns=["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"])
df1[feature_cols] = scaler.transform(df1[feature_cols])
df1["Volume"] = scaler1.transform(df1[["Volume"]])
df1

,Date,Ticker,Open,High,Low,Close,Volume
0,2023-01-03,AAPL,0.623278,0.612889,0.569116,0.561800,3.697717
1,2023-01-04,AAPL,0.583652,0.586973,0.579894,0.576898,2.801964
2,2023-01-05,AAPL,0.586458,0.576676,0.576104,0.561215,2.484575
3,2023-01-06,AAPL,0.573366,0.605831,0.577643,0.615052,2.749050
4,2023-01-09,AAPL,0.625499,0.641928,0.636862,0.621256,2.088489
...,...,...,...,...,...,...,...
1995,2023-12-22,XOM,0.259961,0.253179,0.267733,0.254458,-0.164882
1996,2023-12-26,XOM,0.264835,0.254189,0.271178,0.257068,-0.012501
1997,2023-12-27,XOM,0.256901,0.248805,0.262221,0.251621,-0.101139
1998,2023-12-28,XOM,0.249535,0.238260,0.248326,0.234940,-0.032197


In [6]:
stocks = ['AAPL', 'AMZN', 'BA', 'BAC', 'C', 'CAT', 'CVX', 'DUK', 'GOOGL', 'JNJ', 'JPM' ,'KO', 'MRK', 'MSFT', 'PFE', 'PG', 'T', 'VZ', 'WMT', 'XOM']

ticker_to_idx = {ticker: idx for idx, ticker in enumerate(sorted(tickers))}
print("Ticker to index mapping:", ticker_to_idx)

Ticker to index mapping: {'AAPL': 0, 'AMZN': 1, 'BA': 2, 'BAC': 3, 'C': 4, 'CAT': 5, 'CVX': 6, 'DUK': 7, 'GOOGL': 8, 'JNJ': 9, 'JPM': 10, 'KO': 11, 'MRK': 12, 'MSFT': 13, 'PFE': 14, 'PG': 15, 'T': 16, 'VZ': 17, 'WMT': 18, 'XOM': 19}


In [7]:
hyperedge_indices = []
for he_name, ticker in hyperedges.items():
    indices = [ticker_to_idx[t] for t in ticker]
    print(f"Hyperedge '{he_name}': {ticker} -> {indices}")
    hyperedge_indices.append(indices)

Hyperedge 'Tech': ['AAPL', 'GOOGL', 'MSFT', 'AMZN'] -> [0, 8, 13, 1]
Hyperedge 'Finance': ['JPM', 'BAC', 'C'] -> [10, 3, 4]
Hyperedge 'Healthcare': ['JNJ', 'PFE', 'MRK'] -> [9, 14, 12]
Hyperedge 'Energy': ['XOM', 'CVX'] -> [19, 6]
Hyperedge 'Consumer': ['WMT', 'PG', 'KO'] -> [18, 15, 11]
Hyperedge 'Industrials': ['BA', 'CAT'] -> [2, 5]
Hyperedge 'Communications': ['VZ', 'T'] -> [17, 16]
Hyperedge 'Utilities': ['DUK'] -> [7]


In [8]:
adj_matrix = [[0 for i in range(20)] for j in range(20)]
for hyperedge in hyperedge_indices:
    for i in range(len(hyperedge)):
        for j in range(i+1, len(hyperedge)):
            adj_matrix[hyperedge[i]][hyperedge[j]] = 1
            adj_matrix[hyperedge[j]][hyperedge[i]] = 1
adj_matrix = np.array(adj_matrix)
adj_matrix

array([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
import torch
edge_index = torch.tensor(np.array(np.nonzero(adj_matrix)), dtype=torch.long)
edge_index

In [10]:
def encode_date(date_str):
    date = pd.to_datetime(date_str)
    return [
        date.month / 12,
        date.day / 31,
        date.weekday() / 6,
        np.sin(2 * np.pi * date.dayofyear / 365.25),
        np.cos(2 * np.pi * date.dayofyear / 365.25),  
    ]

In [11]:
def get_stock_features(stock, date):
    stock_idx = ticker_to_idx[stock]
    date_features = encode_date(date) 
    return stock_idx, date_features

In [46]:
def get_features_for_date(date, df, tickers_today):
    stock_feats = []
    targets = []
    for stock in tickers_today:
        row = df[(df['Date'] == date) & (df['Ticker'] == stock)]
        if row.empty:
            continue
        stock_idx, date_feat = get_stock_features(stock, date)
        x = date_feat
        y = row[['Open', 'High', 'Low', 'Close', 'Volume']].values[0]
        stock_feats.append(x)
        targets.append(y)
    return torch.tensor(stock_feats, dtype=torch.float), torch.tensor(targets, dtype=torch.float)


In [47]:
def mask_edge_index(edge_index, current_tickers, all_tickers):
    idx_map = {ticker: i for i, ticker in enumerate(all_tickers)}
    local_map = {ticker: i for i, ticker in enumerate(current_tickers)}
    
    keep_edges = []
    for i in range(edge_index.shape[1]):
        src_global = edge_index[0, i].item()
        dst_global = edge_index[1, i].item()
        src_ticker = all_tickers[src_global]
        dst_ticker = all_tickers[dst_global]
        if src_ticker in current_tickers and dst_ticker in current_tickers:
            keep_edges.append((local_map[src_ticker], local_map[dst_ticker]))

    if not keep_edges:
        return torch.empty((2, 0), dtype=torch.long)
    
    edge_index_masked = torch.tensor(keep_edges, dtype=torch.long).t().contiguous()
    return edge_index_masked


In [53]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
import torch.nn.functional as F


class GATStockPredictor(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=64, out_dim=5, heads=16):
        super().__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1)
        self.regressor = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, edge_index):
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))
        return self.regressor(x)


In [73]:
from tqdm import tqdm

def train(model, optimizer, loss_fn, train_dates, val_dates, edge_index_full, df, df1, all_tickers, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        with tqdm(train_dates, desc=f"Epoch {epoch+1} [Train]") as t:
            for date in t:
                tickers_today = sorted(df[df['Date'] == date]['Ticker'].unique())
                if not tickers_today:
                    continue
                x, y = get_features_for_date(date, df, tickers_today)
                edge_index_masked = mask_edge_index(edge_index_full, tickers_today, all_tickers)
                if edge_index_masked.numel() == 0:
                    continue
                
                optimizer.zero_grad()
                x, edge_index_masked, y = x.to(device), edge_index_masked.to(device), y.to(device)
                preds = model(x, edge_index_masked)
                loss = loss_fn(preds, y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                t.set_postfix(loss=loss.item())

        model.eval()
        val_loss = 0
        with torch.no_grad():
            with tqdm(val_dates, desc=f"Epoch {epoch+1} [Val]") as t:
                for date in t:
                    tickers_today = sorted(df1[df1['Date'] == date]['Ticker'].unique())
                    if not tickers_today:
                        continue
                    x, y = get_features_for_date(date, df1, tickers_today)
                    edge_index_masked = mask_edge_index(edge_index_full, tickers_today, all_tickers)
                    if edge_index_masked.numel() == 0:
                        continue
                    x, edge_index_masked, y = x.to(device), edge_index_masked.to(device), y.to(device)
                    preds = model(x, edge_index_masked)
                    loss = loss_fn(preds, y)
                    val_loss += loss.item()
                    t.set_postfix(val_loss=loss.item())

        print(f"\nEpoch {epoch+1} Summary: Train Loss = {train_loss / len(train_dates):.4f}, Val Loss = {val_loss / len(val_dates):.4f}\n")


In [74]:
def build_edge_index(tickers):
    n = len(tickers)
    src = []
    dst = []
    for i in range(n):
        for j in range(n):
            if i != j:
                src.append(i)
                dst.append(j)
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    return edge_index

In [75]:
train_dates = df['Date'].unique()
train_dates = sorted(train_dates)
val_dates = df1['Date'].unique()
train_dates = sorted(train_dates)
model = GATStockPredictor(input_dim=5, hidden_dim=128, out_dim=5).to("cuda")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [76]:
all_tickers = sorted((df['Ticker'].unique()))
edge_index_full = build_edge_index(all_tickers)

train(model, optimizer, loss_fn, train_dates, val_dates, edge_index_full, df, df1, all_tickers, device, num_epochs=10)

Epoch 1 [Val]: 100%|██████████| 250/250 [00:02<00:00, 94.28it/s, val_loss=0.723]



Epoch 1 Summary: Train Loss = 0.7798, Val Loss = 0.5594



Epoch 2 [Val]: 100%|██████████| 250/250 [00:03<00:00, 63.00it/s, val_loss=0.716]



Epoch 2 Summary: Train Loss = 0.7776, Val Loss = 0.5640



Epoch 3 [Val]: 100%|██████████| 250/250 [00:03<00:00, 73.60it/s, val_loss=0.708]


...


Epoch 8 Summary: Train Loss = 0.7778, Val Loss = 0.5303



Epoch 9 [Val]: 100%|██████████| 250/250 [00:02<00:00, 87.52it/s, val_loss=0.706]



Epoch 9 Summary: Train Loss = 0.7783, Val Loss = 0.5344



Epoch 10 [Val]: 100%|██████████| 250/250 [00:03<00:00, 66.78it/s, val_loss=0.709]


Epoch 10 Summary: Train Loss = 0.7786, Val Loss = 0.5345



In [83]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
model = model.to("cpu")
def evaluate(model, val_dates, df, val_edge_index, tickers, device):
    model.eval()
    preds_list, actuals_list = [], []

    with torch.no_grad():
        for date in tqdm(val_dates, desc="Evaluating"):
            tickers_today = sorted(df[df['Date'] == date]['Ticker'].unique())
            if len(tickers_today) == 0:
                continue

            x, y = get_features_for_date(date, df, tickers_today)
            edge_index = mask_edge_index(val_edge_index, tickers_today, tickers)
            if edge_index.numel() == 0:
                continue
            x = torch.tensor(x, dtype=torch.float).to(device)
            y = torch.tensor(y, dtype=torch.float).to(device)
            edge_index = edge_index.to(device)

            x = x.to(device)
            edge_index = val_edge_index.to(device)

            preds = model(x, edge_index)

            preds_list.append(preds.cpu())
            actuals_list.append(y)

    preds = torch.cat(preds_list, dim=0)
    actuals = torch.cat(actuals_list, dim=0)

    mask = actuals.abs().sum(dim=1) > 0
    preds = preds[mask].numpy()
    actuals = actuals[mask].numpy()

    preds[:, :4] = scaler.inverse_transform(preds[:, :4])
    actuals[:, :4] = scaler.inverse_transform(actuals[:, :4])
    preds[:, 4] = scaler1.inverse_transform(preds[:, 4].reshape(-1, 1)).reshape(-1)
    actuals[:, 4] = scaler1.inverse_transform(actuals[:, 4].reshape(-1, 1)).reshape(-1)

    mae = mean_absolute_error(actuals, preds)
    rmse = np.sqrt(mean_squared_error(actuals, preds))
    mape = np.mean(np.abs((actuals - preds) / actuals)) * 100

    print(f"\nMAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}%")
    return mae, rmse, mape

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [84]:
evaluate(model, val_dates, df1, edge_index_full, all_tickers, "cpu")

TypeError: evaluate() takes 5 positional arguments but 6 were given